# Classification using TidyModels

In this lab we would be going through:
- Logistic Regression
- Linear Discriminant Analysis
- Quadratic Discriminant Analysis

using TidyModels. 

For this lab, we would examining the `OJ` data set that contains a number of numeric variables plus a variable called `Purchase` which has the two labels `CH` and `MM` (which is Citrus Hill or Minute Maid Orange Juice)

In [ ]:
suppressPackageStartupMessages(library(tidymodels))
suppressPackageStartupMessages(library(ISLR))
suppressPackageStartupMessages(library(discrim))
suppressPackageStartupMessages(library(corrr))


In [ ]:
head(OJ)

In [ ]:
attach(OJ)

The `correlate()` function (from `corrr` package) will calculate the correlation matrix between all the variables that it is being fed.

In [ ]:
cor_oj <- OJ %>%
  select(-Purchase, -Store7) %>% #Remove Purchase & Store as it not numeric
  correlate()

Lets pass this correlation to `rplot()` to visualize the correlation matrix

In [ ]:
rplot(cor_oj, colours = c("indianred2", "black", "skyblue1"))

## Logistic Regression

Now we will fit a logistic regression model. We will again use the `parsnip` package, and we will use `logistic_reg()` to create a logistic regression model specification.

In [ ]:
lr_spec <- logistic_reg() %>%
  set_engine("glm") %>% #default engine
  set_mode("classification") #default mode

We want to model the `Direction` of the stock market based on the percentage return from the 5 previous days plus the volume of shares traded. 

In [ ]:
lr_fit <- lr_spec %>%
  fit(
    Purchase ~ PriceCH + PriceMM + SalePriceMM + SalePriceCH + WeekofPurchase,
    data = OJ
    )

lr_fit

In [ ]:
lr_fit %>%
  pluck("fit") %>%
  summary()

The `summary()` lets us see a couple of different things such as; parameter estimates, standard errors, p-values, and model fit statistics. 

we can use the `tidy()` function to extract some of these model attributes for further analysis or presentation.

In [ ]:
tidy(lr_fit)

In [ ]:
predict(lr_fit, new_data = OJ)

The result is a tibble with a single column `.pred_class` which will be a factor variable of the same labels as the original training data set.

We can also get back probability predictions, by specifying `type = "prob"`

In [ ]:
predict(lr_fit, new_data = OJ, type = "prob")

We can describe a `confusion matrix` that would help us understand how well the predictive model is preforming by given a table of predicted values against the true value

`augment()` function helps add the predictions to the `data.frame` and then use that to look at model performance metrics.

In [ ]:
augment(lr_fit, new_data = OJ) %>%
  conf_mat(truth = Purchase, estimate = .pred_class)

 We can represent this as a `heatmap`

In [ ]:
augment(lr_fit, new_data = OJ) %>%
  conf_mat(truth = Purchase, estimate = .pred_class) %>%
  autoplot(type = "heatmap")

A good performing model would ideally have high numbers along the diagonal (up-left to down-right) with small numbers on the off-diagonal. We see here that the model isn’t great, as it tends to predict `"CH"` as `"MM"` more often than it should.

We can also calculate various performance metrics. One of the most common metrics is accuracy, which is how often the model predicted correctly as a percentage.

In [ ]:
augment(lr_fit, new_data = OJ) %>%
  accuracy(truth = Purchase, estimate = .pred_class)

Fitting a model and evaluating the model on the same data would give much information abou the model's performance.

Let us instead split up the data, train it on some of it and then evaluate it on the other part of the data. Since we are working with some data that has a time component,lets train the data over a before a specific week and test it over the set of other weeks.

This would more closely match how such a model would be used in real life.

In [ ]:
mean(WeekofPurchase)

In [ ]:
oj_train <- OJ %>%
  filter(WeekofPurchase < 260)
dim(oj_train)

oj_test <- OJ %>%
  filter(WeekofPurchase >= 260)
dim(oj_test)

dim(OJ)

In [ ]:
# Build an lr model that fits Purchase as response with other numeric variables
# Predictors: PriceCH, PriceMM, DiscCH, DiscMM, PctDiscMM, PctDiscCH
# Modeled over the training data set created above
lr.fit = function(){
    # your code here
    
}


In [ ]:
summary = lr.fit() %>% pluck('fit') %>% summary()
coeff = coef(summary)

stopifnot(round(coeff[1],2) == 1.47) #Intercept test case
stopifnot(round(coeff[2],2) == 2.99) #PriceCH test case

In [ ]:
# hidden test cases

In [ ]:
# Return a confusion matrix and accuracy of the model lr.fit 
# the matrix has to be defined over the test data set 
confusion_matrix = function(){
    # your code here
    
}

accuracy.fit = function(){
    # your code here
    
}

In [ ]:
confusion_matrix()


accuracy = accuracy.fit()
stopifnot(round(accuracy[3],2) == 0.70) #Accuracy test case

In [ ]:
# hidden test cases

## Linear Discriminant Analysis

We will use the `discrim_linear()` function to create a LDA specification. We are gonna use two predictors (`PriceCH` & `PriceMM`) for easy comparision

In [ ]:
lda_spec <- discrim_linear() %>%
  set_mode("classification") %>%
  set_engine("MASS")

In [ ]:
lda_fit = lda_spec %>%
  fit(Purchase ~ PriceCH + PriceMM, data = oj_train)

lda_fit

In [ ]:
predict(lda_fit, new_data = oj_test)

In [ ]:
#confusion matrix
augment(lda_fit, new_data = oj_test) %>%
  conf_mat(truth = Purchase, estimate = .pred_class)

#accuracy 
augment(lda_fit, new_data = oj_test) %>%
      accuracy(truth = Purchase, estimate = .pred_class)

Lets compare this to `lr()` fit

In [ ]:
lda_fit_2 = lda_spec %>%
  fit(Purchase ~ PriceCH + PriceMM + DiscCH + DiscMM + PctDiscMM + PctDiscCH,
      data = oj_train)

#accuracy
augment(lda_fit_2, new_data = oj_test) %>%
      accuracy(truth = Purchase, estimate = .pred_class)

## Quadratic Discriminant Analysis

We can fit a `QDA` model by using the `discrim_quad()` function. 

In [ ]:
qda_spec = discrim_quad() %>%
  set_mode("classification") %>%
  set_engine("MASS")

`qda_spec` has a similar usage as `lda_spec`. so, 

In [ ]:
qda_fit = qda_spec %>% fit(Purchase ~ PriceCH + PriceMM, 
                           data = oj_train)
qda_fit

In [ ]:
#confusion matrix
augment(qda_fit, new_data = oj_test) %>%
  conf_mat(truth = Purchase, estimate = .pred_class) 

#accuracy
augment(qda_fit, new_data = oj_test) %>%
  accuracy(truth = Purchase, estimate = .pred_class) 

We can see that, `QDA` performs better compared to `LDA` using two predictors

Now lets compare all the three fits with 6 predictors

In [ ]:
qda_fit_2 = qda_spec %>%
  fit(Purchase ~ PriceCH + PriceMM + DiscCH + DiscMM + PctDiscMM + PctDiscCH,
      data = oj_train)

#accuracy
augment(qda_fit_2, new_data = oj_test) %>%
      accuracy(truth = Purchase, estimate = .pred_class)

In [ ]:
get_accuracy = function(fit){
    accuracy = augment(fit, new_data = oj_test) %>% 
      accuracy(truth = Purchase, estimate = .pred_class)
    accuracy[3]
}
accuracy_matrix = matrix(c( get_accuracy(lr.fit())
                           , get_accuracy(lda_fit_2)
                           , get_accuracy(qda_fit_2))
                           , nrow = 1, ncol = 3, byrow=FALSE)
colnames(accuracy_matrix) = c("LR", "LDA", "QDA")
accuracy_matrix

we can see that the `QDA` works better for this data where as `LDA` and `LR` work similary for this data set